In [ ]:
%matplotlib inline
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from IPython.display import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
from metpy.units import units
import numpy as np
from siphon.ncss import NCSS
import metpy.calc as mpcalc
import xarray as xr


In [ ]:
def getdata(year='1999',month='6', day='19', hour='12'):
    
    # Reduce lat/lon to get rid of missing values
    iymin = 5
    ixmin = 5
    iymax = 270
    ixmax = 335
    
    dt = datetime(int(year),int(month),int(day),int(hour))
    
    # Grab Pressure Level Data
    air_data = xr.open_dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                               f'NARR/pressure/air.{dt:%Y%m}.nc')
    uwnd_data = xr.open_dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                f'NARR/pressure/uwnd.{dt:%Y%m}.nc')
    vwnd_data = xr.open_dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                f'NARR/pressure/vwnd.{dt:%Y%m}.nc')
    shum_data = xr.open_dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                f'NARR/pressure/shum.{dt:%Y%m}.nc')
    hght_data = xr.open_dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                f'NARR/pressure/hgt.{dt:%Y%m}.nc')
        
    vtimes = air_data.time.sel(time=dt).values.astype('datetime64[ms]').astype('O')
      
    data = {}
    data['hght'] = hght_data.hgt.sel(time=dt).isel(y=slice(iymin,iymax), x=slice(ixmin, ixmax)).metpy.unit_array
    data['airtemp'] = air_data.air.sel(time=dt).isel(y=slice(iymin,iymax), x=slice(ixmin, ixmax)).metpy.unit_array
    data['uwnd'] = uwnd_data.uwnd.sel(time=dt).isel(y=slice(iymin,iymax), x=slice(ixmin, ixmax)).metpy.unit_array
    data['vwnd'] = vwnd_data.vwnd.sel(time=dt).isel(y=slice(iymin,iymax), x=slice(ixmin, ixmax)).metpy.unit_array
    data['shum'] = shum_data.shum.sel(time=dt).isel(y=slice(iymin,iymax), x=slice(ixmin, ixmax)).metpy.unit_array
    data['time'] = vtimes
    data['level'] = air_data.level.values
    data['lat'] = air_data.lat.isel(y=slice(iymin,iymax), x=slice(ixmin, ixmax)).values
    data['lon'] = air_data.lon.isel(y=slice(iymin,iymax), x=slice(ixmin, ixmax)).values
    
    return data

In [ ]:
def plot(level='300'):
    data = x.widget.result
    year = x.widget.kwargs['year']
    month = x.widget.kwargs['month']
    day = x.widget.kwargs['day']
    hour = x.widget.kwargs['hour']
    
    
    dt = datetime(int(year),int(month),int(day),int(hour))
    
    # Pull out the lat and lon data
    lat = data['lat'][:]
    lon = data['lon'][:]

    plotcrs = ccrs.LambertConformal(central_latitude=45., central_longitude=-100.,
                                    standard_parallels=[30, 60])
    
    datacrs = ccrs.PlateCarree()

    tlatslons = plotcrs.transform_points(datacrs,lon,lat)
    tlon = tlatslons[:,:,0]
    tlat = tlatslons[:,:,1]
    
    preslevs = units('hPa') * data['level'][:]
    temp_var = mpcalc.smooth_n_point(data['airtemp'][:], 9, 10)
    shum_var = mpcalc.smooth_n_point(data['shum'][:], 9, 10)
    uwnd_var = mpcalc.smooth_n_point(data['uwnd'][:], 9, 10)
    vwnd_var = mpcalc.smooth_n_point(data['vwnd'][:], 9, 10)
    #hght_var = data['Geopotential_Height'][:] * units.meter
    
    isentlevs = [float(level)] * units.kelvin
    isent_anal = mpcalc.isentropic_interpolation(isentlevs,
                                                 preslevs,
                                                 temp_var,
                                                 shum_var,
                                                 uwnd_var,
                                                 vwnd_var,
                                                 temperature_out=True)
    
    isent_pres = isent_anal[0]
    isent_temp = isent_anal[1]
    isent_shum = isent_anal[2]
    isent_uwnd = isent_anal[3]
    isent_vwnd = isent_anal[4]
    #isent_hght = isent_anal[5]
    
    e = mpcalc.vapor_pressure(isent_pres[0],isent_shum[0])
    es = mpcalc.saturation_vapor_pressure(isent_temp[0])
    rel_hum = e/es*100
    rel_hum[rel_hum>100] = 100
    
    fig = plt.figure(figsize=(16, 14))
    ax = fig.add_subplot(111, projection=plotcrs)
    ax.set_extent([230., 290., 20., 55.], ccrs.PlateCarree())
    #ax.set_title('NARR Plot', size=16)

    # Add state boundaries to plot
    ax.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='black', linewidth=1)

    # Add country borders to plot
    ax.add_feature(cfeature.BORDERS.with_scale('50m'), edgecolor='black', linewidth=1)
        
    # Contour based on variable chosen
    clevisent = np.arange(0, 1001, 50)
    cs = ax.contour(tlon, tlat, isent_pres[0],clevisent,
                    linewidth=2, colors='black')
    plt.clabel(cs,inline=True,fmt='%d')
        
    cf = ax.contourf(tlon, tlat, rel_hum, range(50,101,5), cmap='Greens', norm=plt.Normalize(50, 120))
    plt.title(r''+str(level)+'K Isentropic Pressure (hPa), Relative Humidity (%), and Wind Barbs (kts)',
              loc='left')
    
    cb = fig.colorbar(cf, orientation='horizontal',ax=ax,pad=0,aspect=50,extendrect='True')
        
    # Plot Wind Barbs
    # Transform Vectors and plot wind barbs.
    wind_slice = [slice(None, None, 7), slice(None, None, 7)]
    ax.barbs(lon[wind_slice], lat[wind_slice],
             isent_uwnd[0][wind_slice].to('kts').m, isent_vwnd[0][wind_slice].to('kts').m,
             length=6, pivot='middle',
             transform=datacrs)

    
    plt.title(f'VALID: {data["time"]}', loc='right')
    
    plt.show()



In [ ]:
year_widget = widgets.Dropdown(
    options=list(range(1979,2017)),
    description='Year', alignment='center')

month_widget = widgets.Dropdown(
    options=list(range(1,13)),
    description='Month', alignment='center')

day_widget = widgets.Dropdown(description='Day', options=list(range(1,32)))
hour_widget = widgets.Dropdown(description='Hour', options=[0,3,6,9,12,15,18,21])


level_widget = widgets.BoundedIntText(value=300,min=255,max=385,step=1,
                                      description='Level: ',disabled=False)


In [ ]:
x = widgets.interact_manual(getdata,year=year_widget,month=month_widget, day=day_widget, hour=hour_widget)

In [ ]:
print('Input the desired Isentropic Level (valid level: 255-385K)')
p = widgets.interact_manual(plot, level=level_widget)